<h1>Merkle trees</h1>

> In cryptography and computer science, a hash tree or Merkle tree is a tree in which every "leaf" (node) is labelled with the cryptographic hash of a data block, and every node that is not a leaf (called a branch, inner node, or inode) is labelled with the cryptographic hash of the labels of its child nodes. A hash tree allows efficient and secure verification of the contents of a large data structure. A hash tree is a generalization of a hash list and a hash chain.  
-- <cite>Source: <a href="https://en.wikipedia.org/wiki/Merkle_tree">Wikipedia: Merkle Tree</a></cite>

<figure>
    <img src="https://upload.wikimedia.org/wikipedia/commons/9/95/Hash_Tree.svg" alt="Example of a merle tree">
    <figcation>An example of a binary hash tree. Hashes 0-0 and 0-1 are the hash values of data blocks L1 and L2, respectively, and hash 0 is the hash of the concatenation of hashes 0-0 and 0-1.</figcaption>  
    
-- <cite>Source: <a href="https://en.wikipedia.org/wiki/Merkle_tree#/media/File:Hash_Tree.svg">Hash Tree</a></cite>
</figure>

We will be using binary trees which means we will need to provide an implementation for the nodes first. The nodes can have either 0, 1 or 2 children, but one question that might arise is how de we deal with an odd number of leafs (or and odd number of branches). One way to deal with that is by duplicating the last node.  
Thus we will make sure that the right child of a node (if not provided) will be the same as its left child.  

Merkle tree nodes can be instanciated either by providing one or both of the children nodes; passed as keyword arguments only, or by providing the value of the node only.  
I prefered it this way instead of passing `(hash(left) + hash(right), left, right)` to the constructor for ease of use and to avoid any unexpected errors, I think its better to let the implementation take care of adding the hashes of the children.  

I chose to keep all the values of a node's children just for inspection.  

Adding two hash values in this context means concatenating them, so one needs to be carful not to concatenate the hex representations of the hashes (using `hexdigest()` which returns the hex representation of the hash as a string) but instead use their bytes representation (using `digest()`).  
We will be using SHA-256 hashing function by the way.

I chose to make this class private since it is not meant to be instanciated by users, it is just a helper class for creating the tree, that is why I also chose to break the tradition of making a class's `__repr__` return a string which can be passed to `eval()` (or copied and executed on an interactive session) to recreate an instance, and instead make it return user friendly string representation of the instance. This would help later on, as you'll see, when displaying a list of nodes (since lists use an object's `__repr__` instead of `__str__`).  

I've also added a `__eq__` method which would be useful later for comparing between trees (in unit testing).

In [ ]:
from hashlib import sha256

class _MerkleNode:
    def __init__(self, value=None, *, left=None, right=None):
        if value is not None and not (left is None and right is None):
            raise ValueError("Can't provide both value and left or right")

        if right is None:
            right = left
        self.left = left
        self.right = right
        if value is None:
            self._hash = sha256(left._hash.digest() + right._hash.digest())
            value = left.value + right.value
        else:
            self._hash = sha256(value.encode("utf-8"))
            value = [value]
        self.value = value

    def __eq__(self, other):
        if type(other) is not type(self):
            return NotImplemented
            
        return (
            (self.value, self.left, self.right, self.hash) ==
            (other.value, other.left, other.right, other.hash)
            )

    @property
    def hash(self):
        return self._hash.hexdigest()

    def __repr__(self):
        return f"{self.value}: {self.hash}"


Now we will move to implementing the tree itself.
The steps are the following:  
> 1. Find the hash value of every data item. These hash values will be the leaf nodes of our Merkle Tree.
> 2. Group the leaf nodes into pairs, starting from left to right.
> 3. Create a parent node for each pair. Combine the hash values of the leaf nodes and apply the hash function to the combined value. This will be the label of the parent node.
> 4. Next group the parent nodes into pairs and repeat the above step.
> 5. Continue the pairing until we are left with a single node which is the Merkle root or root hash.  
> What if we need to pair an odd number of nodes? Just create a duplicate of the last node. We now have an even number of nodes in the Merkle Tree.  
> <cite>-- Source: <a href="https://kba.ai/merkle-tree-a-beginners-guide/">Merkle Tree: A Beginners Guide</a></cite>  

I have added an extra private class `_NO_ROOT` to handle the case where the constructor is given an empty list, that is the result is an empty tree. I did this to give a meaningful type to the root of an empty tree instead of `None` and better exceptions than the generic: 
```py
>>> tree = MerkleTree([])
>>> tree.root.value
AttributeError: 'NoneType' object has no attribute 'value'
>>> tree.root.hash
AttributeError: 'NoneType' object has no attribute 'hash'
>>> type(tree.root)
<class 'NoneType'>
```

The trick for getting 2 successive elements at a time from an iterable can be found here: <a href="https://docs.python.org/3/library/itertools.html#itertools-recipes">Itertools recipes</a>. (Look for the grouper function).  
I chose to return the nodes as a list of lists where each sublist contains the nodes of a whole level in the tree, this was done using a breadth first instead of a depth first approach, for more informations check: <a href="https://www.codecademy.com/article/tree-traversal">Tree traversal</a>

In [ ]:
from itertools import zip_longest
from queue import Queue


class _NO_ROOT:
    
    def __str__(self):
        return "<Empty tree: Not root>"

    @property
    def value(self):
        raise AttributeError("Empty tree: root does not exist")
    
    @property
    def hash(self):
        raise AttributeError("Empty tree: root does not exist")


class MerkleTree:

    NO_ROOT = _NO_ROOT()

    def __init__(self, blocks):
        self.blocks = blocks
        self.root = self.NO_ROOT
        self._build_tree([_MerkleNode(block) for block in self.blocks])

    @property
    def nodes(self):
        if self.root is self.NO_ROOT:
            return None
        
        nodes = Queue()
        nodes.put(self.root)
        childs = []
        temp = []
        i = 0
        level_nodes = 1
        while not nodes.empty():
            i += 1
            node = nodes.get()
            temp.append(node)
            if i == level_nodes:
                childs.append(temp)
                temp = []
                i = 0
                level_nodes *= 2
            left, right = node.left, node.right
            if left:
                nodes.put(left)
            if right:
                nodes.put(right)
        
        return childs
    
    def _build_tree(self, children):
        if len(children) == 0:
            return
        if len(children) == 1:
            self.root = children[0]
            return

        parents = []
        children = [iter(children)] * 2
        for left_child, right_child in zip_longest(*children):
            parent = _MerkleNode(left=left_child, right=right_child)
            parents.append(parent)
        return self._build_tree(parents)


Demo:

In [ ]:
tree = MerkleTree(["ahmed", "amine", "yousa"])
print("Root:", tree.root)
print("Root's values:",tree.root.value)
print("Root's hash:", tree.root.hash)
print("Nodes per level:")
for level, nodes in enumerate(tree.nodes):
    print("level={}\n[\n  {}\n]".format(level, ',\n  '.join(str(node) for node in nodes)))
    